In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
result_path = '/home/yifanc3/segmentation/DEM_segmentation/results/erosion/128over_unet_sigmoid_40e/'
X =  np.load(result_path + 'inputs.npy')
Y_true = np.load(result_path + 'gt_labels.npy')
Y_pred = np.load(result_path + 'epoch40/pred_labels.npy')

In [19]:
from skimage.morphology import skeletonize


def centerline_acc(y_true, y_pred):
    """
    acc = ( y_true_center & y_pred ) / y_true_center
    """
    smooth = 0.01
    y_pred = (y_pred >= 0.5).astype('uint8')
    y_true = y_true.astype('uint8')
    n = len(y_true)
    acc = 0
    for i in range(n):
        y_pred_curr = np.squeeze(y_pred[i])
        y_true_curr = np.squeeze(y_true[i])
        
        
        y_true_center = skeletonize(y_true_curr).astype('uint8')     
        tmp = np.sum(y_true_center&y_pred_curr)/(np.sum(y_true_center) + smooth)

        y_pred_center = skeletonize(y_pred_curr).astype('uint8')
        tmp2 = np.sum(y_pred_center&y_true_curr)/(np.sum(y_pred_center) + smooth)

        if(np.sum(y_true_center)<10 or np.sum(y_pred_center)<10): # if there is too little features in an image, ignore it
            n-=1
            continue
            
        print(tmp, tmp2)
        
        acc += (tmp + tmp2)/2

    return acc/n

In [20]:
centerline_acc(Y_true,Y_pred)

0.0481869654258523 0.1162520344106022
0.11109988890011109 0.2579812963560142
0.0 0.0
0.629142792944613 0.9331778036993834
0.5280305583642287 0.6817148916830783
0.06097189195780745 0.365764447695684
0.4649274934504262 0.7437035185300919
0.43134435657800146 0.999857163262391
0.0 0.0
0.5356186395286556 0.9711510996858042
0.5369376041473801 0.9996297667530544
0.22573363431151242 0.31562335612835346
0.5804579168010319 0.9992862241256246
0.9793919608243216 0.9613535858488752
0.38767598449296065 0.222194790766572
0.7245326764237067 0.5090446323061539
0.326508400789062 0.9421265141318977
0.3398470688190315 0.9998148491020182
0.08916629514043692 0.30945013092120927
0.1479754271108919 0.4285204142364004
0.208618085029854 0.5150734737161036
0.2753224170410085 0.47044986768597474
0.06541444724792075 0.24989587671803415
0.0 0.0
0.47990401919616077 0.8617718028266115
0.5198960207958409 0.39387971519466747
0.11536982438148954 0.2665778073975341
0.21083067285103307 0.6778512116590408
0.931712305419838

0.41316199556912203

In [10]:
from scipy import signal
import numpy as np
def terrain_analysis(array, size):
    """calculate terrain derivatives based on the Evans Young method
    Args:
      array (ndarray): elevation data array
      size (float,float): size of sample in projected coordinates
    Returns:
      (ndarray): 3d array with original elevation data and derivatives
    """

    px, py = size[0]/array.shape[-1], size[1]/array.shape[-2]

    g = [[(-1/(6*px)), 0 , (1/(6*px))],
         [(-1/(6*px)), 0 , (1/(6*px))],
         [(-1/(6*px)), 0 , (1/(6*px))]]
    h = [[(1/(6*py)),(1/(6*py)),(1/(6*py))],
         [0,0,0],
         [(-1/(6*py)),(-1/(6*py)),(-1/(6*py))]]
    d = [[(1/(3*(px**2))),(-2/(3*(px**2))),(1/(3*(px**2)))],
         [(1/(3*(px**2))),(-2/(3*(px**2))),(1/(3*(px**2)))],
         [(1/(3*(px**2))),(-2/(3*(px**2))),(1/(3*(px**2)))]]
    e = [[(1/(3*(py**2))),(1/(3*(py**2))),(1/(3*(py**2)))],
         [(-2/(3*(py**2))),(-2/(3*(py**2))),(-2/(3*(py**2)))],
         [(1/(3*(py**2))),(1/(3*(py**2))),(1/(3*(py**2)))]]
    f = [[(-1/(4*(px*py))),0, (1/(4*(px*py)))],
         [0,0,0],
         [(1/(4*(px*py))),0,(-1/(4*(px*py)))]]

    gi = signal.convolve2d(array, g, boundary='symm', mode='same')
    hi = signal.convolve2d(array, h, boundary='symm', mode='same')
    di = signal.convolve2d(array, d, boundary='symm', mode='same')
    ei = signal.convolve2d(array, e, boundary='symm', mode='same')
    fi = signal.convolve2d(array, f, boundary='symm', mode='same')

    slope  = np.sqrt (np.power(hi,2)+np.power(gi,2))
    aspect = np.arctan(hi/gi)
    planc  = -1*((np.power(hi, 2)*di)-(2*gi*hi*fi)+(np.power(gi,2)*ei)/(np.power((np.power(gi,2)+np.power(hi,2)),1.5)))
    profc  = -1*(((np.power(gi,2)*di)+(2*gi*hi*fi) +(np.power(hi,2)*ei))/ ((np.power(gi,2)+np.power(hi,2))*(np.power( (1+np.power(gi,2)+np.power(hi,2)),1.5)) ))
    meanc  = -1 *( ((1+np.power(hi,2))*di) -(2*gi*hi*fi) +((1+np.power(gi,2))*ei) / (2*np.power( (1+np.power(gi,2)+np.power(hi,2)),1.5)  ))

    return np.stack([array, slope, aspect, planc, profc,meanc], axis=-1)

In [11]:
print('input.shape',X.shape)
out = terrain_analysis(X[0,:,:,0],(3,3))

input.shape (377, 128, 128, 1)


In [12]:
out.shape

(128, 128, 6)

In [3]:
import richdem as rd
rd.TerrainAttribute(X, attrib='slope_riserun')

ModuleNotFoundError: No module named 'richdem'